In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)


In [3]:
train = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\Pinky\Downloads\pnenomia\chest_xray\train",  # Path to images
    validation_split=0.2,  # 80% training, 20% validation
    subset='training',
    seed=123,
    image_size=(64, 64),
    color_mode='grayscale',                                                        
    batch_size=32
)

validation = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\Pinky\Downloads\pnenomia\chest_xray\train",  # Path to images
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(64, 64),
    color_mode='grayscale',
    batch_size=32
)


Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


# modeling

In [5]:
print(validation.class_names)


['NORMAL', 'PNEUMONIA']


In [6]:
from keras.models import Sequential
classifier = Sequential()

 **step 1 : convolution layer**

In [8]:
from keras.layers import Conv2D
classifier.add(Conv2D(input_shape=[64,64,1],
                      filters=32,kernel_size = 3,activation="relu"))
classifier.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             320 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320 (1.25 KB)

 Trainable params: 320 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

**max pooling**

In [10]:

from keras.layers import MaxPooling2D
classifier.add(MaxPooling2D(pool_size=2,strides=2))

**flattening**

In [12]:
from keras.layers import Flatten
classifier.add(Flatten())


**connection**

In [14]:
from keras.layers import Dense
classifier.add(Dense(units=128,activation = "relu"))
classifier.add(Dense(units =1,activation = "sigmoid"))

In [15]:
classifier.compile(optimizer="adam", 
                   loss = "binary_crossentropy",
                   metrics= ["accuracy"])

In [16]:
classifier.fit(train,validation_data = validation,epochs = 25)

Epoch 1/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.7725 - loss: 53.7098 - val_accuracy: 0.9338 - val_loss: 0.6974
Epoch 2/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - accuracy: 0.9356 - loss: 0.6832 - val_accuracy: 0.9569 - val_loss: 0.4798
Epoch 3/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - accuracy: 0.9542 - loss: 0.5328 - val_accuracy: 0.9722 - val_loss: 0.2693
Epoch 4/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 92ms/step - accuracy: 0.9680 - loss: 0.2708 - val_accuracy: 0.9559 - val_loss: 0.5217
Epoch 5/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.9759 - loss: 0.1713 - val_accuracy: 0.9722 - val_loss: 0.3262
Epoch 6/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - accuracy: 0.9872 - loss: 0.0618 - val_accuracy: 0.9732 - val_loss: 0.3231
Epoch 7/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - accuracy: 0.9968 - loss: 0.0096 - val_accuracy: 0.9741 - val_loss: 0.3156
Epoch 8/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step - accuracy: 0.9981 - loss: 0.0056 -

In [17]:
classifier.save('pnemonia.keras')
# later…
from tensorflow import keras
model = keras.models.load_model('pnemonia.keras')



In [18]:
import numpy as np
from PIL import Image

In [19]:
test_image = Image.open(r"C:\Users\Pinky\Downloads\pnenomia\chest_xray\test\PNEUMONIA\person96_bacteria_466.jpeg")

test_image = test_image.resize((64,64))
test_image = np.array(test_image)
test_image = np.expand_dims(test_image,axis=0)


result = classifier.predict(test_image)

if result[0][0] == 1:
    print("pnemonia")
else:
    print("normal")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
pnemonia


In [20]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\Pinky\Downloads\pnenomia\chest_xray\test\PNEUMONIA",
    labels=None,               # you just want the images
    label_mode=None,           # no labels
    batch_size=32,
    image_size=(64,64),
    color_mode='grayscale',                                                          # same size as training
    shuffle=False              # crucial to keep file order
)
  

Found 390 files.


In [21]:
result = classifier.predict(test_ds)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


In [22]:
result

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.000000

In [23]:
test = tf.keras.preprocessing.image_dataset_from_directory(r"C:\Users\Pinky\Downloads\pnenomia\chest_xray\test\NORMAL",
    labels=None,               # you just want the images
    label_mode=None,           # no labels
    batch_size=32,
    image_size=(64,64),
    color_mode='grayscale',                                                          # same size as training
    shuffle=False              # crucial to keep file order
)
  

Found 234 files.


In [24]:
r = classifier.predict(test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step


In [25]:
r

array([[9.9994117e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999994e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.7774619e-02],
       [1.0000000e+00],
       [9.9942875e-01],
       [9.9999964e-01],
       [2.5737038e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9998444e-01],
       [1.0000000e+00],
       [9.9999875e-01],
       [9.9998802e-01],
       [9.6886998e-01],
       [1.1231037e-16],
       [9.9999964e-01],
       [1.0000000e+00],
       [9.9999988e-01],
       [7.2397995e-01],
       [9.9975646e-01],
       [1.0000000e+00],
       [9.9999934e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9999827e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.9988323e-01],
       [1.0000000e+00],
       [9.5164609e-01],
       [4.7565970e-22],
       [1.4522274e-03],
       [3.8365918e-04],
       [6.7616953e-03],
       [1.0000000e+00],
       [4.721202

In [57]:
import numpy as np

# preds is your (N,1) array of probabilities
r= (r < 0.5).astype(int).ravel()


In [59]:
r

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [61]:
import numpy as np

# preds is your (N,1) array of probabilities
result = (result > 0.5).astype(int).ravel()


In [29]:
result

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,